<a href="https://colab.research.google.com/github/markmaloba/DSCORE-Week-7-Independent-Project/blob/master/DSCORE_W7_INDEPENDENT_PROJECT_Mark_Maloba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Context

As a Data Scientist, you work for Hass Consulting Company which is a real estate leader with over 25 years of experience. You have been tasked to study the factors that affect housing prices using the given information on real estate properties that was collected over the past few months. Later onwards, create a model that would allow the company to accurately predict the sale of prices upon being provided with the predictor variables. 

## Experimental design undertaken

## Exploring the dataset

Here we will seek to understand the dataset, check for data types, dimensionality, missing data, correlation etc

In [0]:
# Import necessary libraries

# import libraries
import numpy as np
import pandas as pd
import seaborn as sb
import scipy as sp

from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

In [2]:
## --PERSONAL NOTE-- ##
#Newer versions of matplotlib have broken Seaborn. Until it gets fixed, you gotta downgrade son.


!pip install matplotlib==3.1.0

     |████████████████████████████████| 13.1MB 280kB/s 
ERROR: plotnine 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 3.1.0 which is incompatible.
ERROR: mizani 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 3.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: matplotlib 3.1.3
    Uninstalling matplotlib-3.1.3:
      Successfully uninstalled matplotlib-3.1.3


In [3]:
# Load the dataset

housedf = pd.read_csv ('house_data.csv')
housedf2 = housedf

housedf.head(5)

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


---
**Dataset glossary**

price  - Price of the house

bedrooms - Number of Bedrooms

bathrooms - Number of Bathrooms

sqft_living - Square feet area of living area

sqft_lot  - Square feet area of parking Layout

floors - Number of Floors

waterfront - Whether waterfront is there or not

view - Number of Views

grade - Grades

sqft_basement - Square feet area off basement

yr_built - Year the house is built

yr_renovated - Year the house is renovated

zipcode - zipcode os the house

lat : Latitude of the house

lon : Longitude of the house

---

## Understanding the dataset

In [4]:
# Checking shape, data types

housedf.shape

(21613, 20)

In [5]:
housedf.dtypes

id                 int64
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

It seems odd that the _floors_ and _bathrooms_ columns would be of data type 'float'. However, I know that in real estate, bathrooms can be counted as a half if they only have a toilet with no shower. Not sure what that is for floors, but I'll generate a few random records to explore and get a better idea before proceeding. 

In [6]:
# Generate random columns

housedf.take(np.random.permutation(len(housedf))[:15])

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
17117,7625701045,360000.0,3,1.75,1510,6000,1.0,0,0,3,7,1060,450,1947,0,98136,47.5535,-122.390,1610,6000
1931,5379804150,598800.0,6,4.00,4470,17877,3.0,0,3,3,9,3230,1240,2013,0,98188,47.4514,-122.273,1790,18260
20531,6821102367,547000.0,3,2.50,1570,1452,2.5,0,0,3,9,1240,330,2007,0,98199,47.6480,-122.396,1670,1596
14209,7857004225,355000.0,2,1.75,1620,3640,1.0,0,0,3,7,900,720,1929,0,98108,47.5430,-122.299,1590,5538
13346,1822350180,375000.0,3,2.25,1330,8004,2.0,0,0,3,7,1330,0,1985,0,98034,47.7098,-122.217,1300,7971
1315,7558700030,5300000.0,6,6.00,7390,24829,2.0,1,4,4,12,5000,2390,1991,0,98040,47.5631,-122.210,4320,24619
1577,7888000390,235000.0,3,1.00,1060,7473,1.0,0,0,3,7,1060,0,1959,0,98198,47.3699,-122.309,1320,7912
12991,415100085,350000.0,3,1.00,1050,8583,1.0,0,0,5,6,1050,0,1955,0,98133,47.7451,-122.339,1460,7351
3691,1003400245,179950.0,3,1.00,1130,9907,1.0,0,0,3,7,1130,0,1954,0,98188,47.4362,-122.286,1320,9907
15741,5129000006,280500.0,3,1.00,1220,4541,1.0,0,0,3,7,890,330,1952,0,98108,47.5387,-122.294,1670,3429


Upon closer inspection, indeed there are incomplete (.5) floors. After researching, I can see that it's not an error. A property with 1.5 floors has the master bedroom on one level, and all other rooms on another.

In [7]:
# Checking for null values

housedf.isnull().any()

id               False
price            False
bedrooms         False
bathrooms        False
sqft_living      False
sqft_lot         False
floors           False
waterfront       False
view             False
condition        False
grade            False
sqft_above       False
sqft_basement    False
yr_built         False
yr_renovated     False
zipcode          False
lat              False
long             False
sqft_living15    False
sqft_lot15       False
dtype: bool

In [8]:
# Check for duplicate rows

duplicates = housedf[housedf.duplicated()]
duplicates.shape

(3, 20)

There are 3 duplicate rows, which will now get dropped!

In [0]:
# Drop duplicate rows

housedf = housedf.drop_duplicates()

Our dataset is complete. There are no null values. It's important to note that this doesn't mean that there aren't any errors, say from transcription, or other data recording forms. The data formats, column headers etc all seem OK, and I have a much better understanding of the dataset. I can now perform descriptive analysis 

## Exploratory Data Analysis

In [10]:
housedf.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161000e+04,2.161000e+04,21610.000000,21610.000000,21610.000000,2.161000e+04,21610.000000,21610.000000,21610.000000,21610.000000,21610.000000,21610.000000,21610.000000,21610.000000,21610.000000,21610.000000,21610.000000,21610.000000,21610.000000,21610.000000
mean,4.580161e+09,5.401789e+05,3.370847,2.114739,2079.881212,1.510829e+04,1.494239,0.007543,0.234197,3.409440,7.656779,1788.347894,291.533318,1971.003609,84.322351,98077.945673,47.560049,-122.213910,1986.518695,12769.031976
std,2.876547e+09,3.673876e+05,0.930110,0.770204,918.500299,4.142323e+04,0.539994,0.086523,0.766136,0.650764,1.175500,828.138723,442.596699,29.372639,401.499264,53.505373,0.138572,0.140833,685.425781,27305.972464
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.216125e+05,3.000000,1.750000,1425.500000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.470925,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.619000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.231000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068875e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


I will use Pandas Profiling to perform univariate and bivariate analysis, and check to see how strongly each variable is correlated to all the others.

In [11]:
import pandas_profiling

pandas_profiling.ProfileReport(housedf)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,21
Number of observations,21610
Total Missing (%),0.0%
Total size in memory,3.5 MiB
Average record size in memory,168.0 B
Numeric,20
Categorical,0
Boolean,1
Date,0
Text (Unique),0
Rejected,0


A few observations:
1. What is immediately apparent is that all measurements of square footage (living space, basement, parking lot) are positively correlated with the price. As one goes up, so does the other.
2. Location, which is ususally a key indicator for property value  in the real estate business, doesn't really help us here since the locations we're given are in latitude + longitude format. One could go to extremes by checkig to see what locations the coordinates translate to, and creating clusters for the same, hoping to find that all the properties listed in the dataset are clustered in similar ways. Only then can one use this categorical data as part of the analysis. 
3. Year built/ renovated don't seem too have much bearing in our dataset as well, unlike with real world examples where one may observe that older houses in certain areas, with a certain je ne sais quoi, may attract higher prices

#### Multivariate Analysis

In [0]:
# preprocessing
X = housedf.drop ('price', 1)
y = housedf ['price']

# splitting the data into train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Normalization
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Applying PCA
from sklearn.decomposition import PCA

pca = PCA()
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

##### Checking for correlations (VIF)


In [13]:
corr = housedf.corr()
corr



,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,1.000000,-0.016803,0.001354,0.005093,-0.012239,-0.132101,0.018482,-0.002717,0.011785,-0.023686,0.008093,-0.010857,-0.005084,0.021218,-0.016688,-0.008129,-0.001861,0.020776,-0.002869,-0.138778
price,-0.016803,1.000000,0.308346,0.525140,0.702046,0.089658,0.256793,0.266332,0.397466,0.036397,0.667478,0.605569,0.323849,0.053980,0.126503,-0.053162,0.306922,0.021567,0.585382,0.082457
bedrooms,0.001354,0.308346,1.000000,0.515935,0.576674,0.031699,0.175448,-0.006583,0.079433,0.028422,0.357003,0.477619,0.303077,0.154276,0.018695,-0.152721,-0.008953,0.129474,0.391633,0.029233
bathrooms,0.005093,0.525140,0.515935,1.000000,0.754688,0.087750,0.500669,0.063747,0.187891,-0.124940,0.664990,0.685351,0.283814,0.505997,0.050879,-0.203831,0.024590,0.223047,0.568665,0.087191
sqft_living,-0.012239,0.702046,0.576674,0.754688,1.000000,0.172830,0.353954,0.103820,0.284641,-0.058768,0.762719,0.876600,0.435054,0.318099,0.055314,-0.199435,0.052521,0.240221,0.756420,0.183285
sqft_lot,-0.132101,0.089658,0.031699,0.087750,0.172830,1.000000,-0.005171,0.021601,0.074753,-0.008967,0.113644,0.183527,0.015271,0.053105,0.007677,-0.129611,-0.085680,0.229552,0.144620,0.718556
floors,0.018482,0.256793,0.175448,0.500669,0.353954,-0.005171,1.000000,0.023711,0.029325,-0.263779,0.458151,0.523874,-0.245673,0.489347,0.006156,-0.059025,0.049593,0.125343,0.279861,-0.011250
waterfront,-0.002717,0.266332,-0.006583,0.063747,0.103820,0.021601,0.023711,1.000000,0.401993,0.016652,0.082785,0.072080,0.080585,-0.026159,0.092950,0.030277,-0.014272,-0.041904,0.086469,0.030702
view,0.011785,0.397466,0.079433,0.187891,0.284641,0.074753,0.029325,0.401993,1.000000,0.045836,0.251378,0.167659,0.276996,-0.053229,0.103208,0.084838,0.006049,-0.078560,0.280404,0.072577
condition,-0.023686,0.036397,0.028422,-0.124940,-0.058768,-0.008967,-0.263779,0.016652,0.045836,1.000000,-0.144655,-0.158212,0.174071,-0.361347,-0.060862,0.002962,-0.014971,-0.106512,-0.092854,-0.003424


In [0]:
plt.figure(figsize = (15, 10))
sb.heatmap(X.corr(), annot = True) 
plt.title('Correlation Heatmap')
plt.show()

In [0]:
# This function will calculate VIF and drop highly correlated variables

from statsmodels.stats.outliers_influence import variance_inflation_factor    
def calculate_vif_(X, thresh):
  cols = X.columns
  variables = np.arange(X.shape[1])
  dropped=True
  while dropped:
      dropped=False
      c = X[cols[variables]].values
      vif = [variance_inflation_factor(c, ix) for ix in np.arange(c.shape[1])]
      maxloc = vif.index(max(vif))
      if max(vif) > thresh:
          print('dropping \'' + X[cols[variables]].columns[maxloc] + '\' at index: ' + str(maxloc))
          variables = np.delete(variables, maxloc)
          dropped=True
  print('Remaining variables:')
  print(X.columns[variables])
  return X[cols[variables]]

In [16]:
# X is the 'housedf' dataset, minus the price column, which is what we're trying to predict using our model.
# 4 is the threshold we're setting for VIF

calculate_vif_(X, 4)

/usr/local/lib/python3.6/dist-packages/statsmodels/stats/outliers_influence.py:185: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


dropping 'sqft_living' at index: 3
dropping 'sqft_above' at index: 9
Remaining variables:
Index(['id', 'bedrooms', 'bathrooms', 'sqft_lot', 'floors', 'waterfront',
       'view', 'condition', 'grade', 'sqft_basement', 'yr_built',
       'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15',
       'sqft_lot15'],
      dtype='object')


,id,bedrooms,bathrooms,sqft_lot,floors,waterfront,view,condition,grade,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,3,1.00,5650,1.0,0,0,3,7,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,3,2.25,7242,2.0,0,0,3,7,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,2,1.00,10000,1.0,0,0,3,6,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,4,3.00,5000,1.0,0,0,5,7,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,3,2.00,8080,1.0,0,0,3,8,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,3,2.50,1131,3.0,0,0,3,8,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,4,2.50,5813,2.0,0,0,3,8,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,2,0.75,1350,2.0,0,0,3,7,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,3,2.50,2388,2.0,0,0,3,8,0,2004,0,98027,47.5345,-122.069,1410,1287


LEt's assess the heatmap. Using a threshold of 0.7 fro our heatmap assessment, we can use the heatmap to see which variables are highly correlated:



1.   Bathrooms vs sqft_living
2.   sqft_living vs grade
3. sqft_living vs sqft_above
4. grade vs sqft_above

There's nothing unusual here. Number of bathrooms will go up as square footage goes up.
ALso, assuming the grades are ordinal in nature, it makes sense that the higher grade houses have more square footage. Once again, nothing unusual there.

These results are supported by the output of the custom function  in the cell above the heatmap. It's no surprise that the two features it picked as having VIFs beyond our desired threshold.

All the measures of square footage are tied into almost everything else.
If we choose to remove one variable from each of the pairs listed above, we get the same result as we did with the VIF assessment function.

We can remove the sqft_living and sqft_above features and see how they affect our accuracy, compared to accuracy when we leave them in.




# Modeling
## Multiple Linear Regression

In [0]:
# Subsetting the data
X = housedf[['bedrooms', 'sqft_lot', 'sqft_above', 'condition', 'yr_built','grade','sqft_basement']]
y = housedf['price']

I feel that removing the sqft_living and sqft_above variables is a crazy thing to do, seeing as they're our two most prominent measures of  Square Footage, but the accuracy will tell all.

In [0]:
# Dividing our data into training and test sets
# ---
# 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [19]:
# Training the Algorithm
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [20]:
# coefficients for our test set attributes. 
## PERSONAL NOTE ##
# Coefficient = SLope 

coeff_df = pd.DataFrame(regressor.coef_, X.columns, columns=['Coefficient'])
coeff_df

,Coefficient
bedrooms,-45471.543710
sqft_lot,-0.351572
sqft_above,207.404286
condition,19186.942248
yr_built,-3384.921344
grade,135672.748209
sqft_basement,218.209661


Below are our predictions compared side by side with the actual, expected values

In [21]:
# Making Predictions
# 
y_pred = regressor.predict(X_test)

# To compare the  output values (without sqft_living and sqft_above)
# 
price_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
price_df

,Actual,Predicted
20188,602000.0,342260.514727
7573,320000.0,373414.905415
12873,245000.0,378083.784647
209,464000.0,549113.424868
19155,190000.0,140518.290604
...,...,...
11080,368888.0,424518.621360
18910,734200.0,750301.931671
15838,411000.0,587348.015428
8695,410000.0,666443.556808


Oof! The predictions are waaaay off. As stated before, the 2 features that the VIF function recommended for dropping are the key indicators for square footage, so we cannot possibly make predictions without them. This proves that. 

Let's try it with just the _sqft_above_ feature, which has a lower correlation index compared to _sqft_living_. Modifications are made by adding and subtracting features from X and y above.

In [22]:
# This is the version that includes sqft_above ONLY
# Making Predictions
# 
y_pred = regressor.predict(X_test)

# To compare the  output values 
# 
price_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
price_df

,Actual,Predicted
20188,602000.0,342260.514727
7573,320000.0,373414.905415
12873,245000.0,378083.784647
209,464000.0,549113.424868
19155,190000.0,140518.290604
...,...,...
11080,368888.0,424518.621360
18910,734200.0,750301.931671
15838,411000.0,587348.015428
8695,410000.0,666443.556808


Our results are much better. We can look at the last version where we keep both features. We will do proper accuracy checks later as well.

In [23]:
# This is the version that includes both sqft_above and sqft_living
# Making Predictions
# 
y_pred = regressor.predict(X_test)

# To compare the  output values 
# 
price_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
price_df

,Actual,Predicted
20188,602000.0,342260.514727
7573,320000.0,373414.905415
12873,245000.0,378083.784647
209,464000.0,549113.424868
19155,190000.0,140518.290604
...,...,...
11080,368888.0,424518.621360
18910,734200.0,750301.931671
15838,411000.0,587348.015428
8695,410000.0,666443.556808


In [24]:
# This is the version that includes sqft_living alone

# 
y_pred = regressor.predict(X_test)

# To compare the  output values 
# 
price_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
price_df

,Actual,Predicted
20188,602000.0,342260.514727
7573,320000.0,373414.905415
12873,245000.0,378083.784647
209,464000.0,549113.424868
19155,190000.0,140518.290604
...,...,...
11080,368888.0,424518.621360
18910,734200.0,750301.931671
15838,411000.0,587348.015428
8695,410000.0,666443.556808


Judging just from observation, it seems that we get similar levels of accuracy when both features are included. 

## Evaluating our linear regression model

I will do two versions of these tests, just to see in numbers, the difference between the model when you include one of our variables in question, and when you include both.

In [25]:
# Evaluating the Algorithm, when you include both features
# ---
# 
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 148921.50835837692
Mean Squared Error: 57719331207.42589
Root Mean Squared Error: 240248.47805433834


In [26]:
# Evaluating the Algorithm, when you include ONLY sqft_living
# ---
# 
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 148921.50835837692
Mean Squared Error: 57719331207.42589
Root Mean Squared Error: 240248.47805433834


In [27]:
# Evaluating the Algorithm, when you include ONLY sqft_above
# ---
# 
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 148921.50835837692
Mean Squared Error: 57719331207.42589
Root Mean Squared Error: 240248.47805433834


All three versions described above produce the same MAE, MSE And RMSE. 

Going forward I'll use ONLY _sqft_above_.

In [28]:
# R2 score

from sklearn.metrics import r2_score
m = r2_score(y_test, y_pred)
m

0.6102276905307119

In [0]:
# Plotting the residual plot
# Residuals have been calculated by by substracting the test value from the predicted value
# 

residuals = np.subtract(y_pred, y_test)

# Plotting the residual scatterplot
#
plt.scatter(y_pred, residuals, color='black')
plt.title('Residual Plot')
plt.ylabel('residual')
plt.xlabel('fitted values')
plt.axhline(y = float(residuals.mean()), color='red', linewidth=1)
plt.show()


In [30]:

# Barlett's test for heteroskedasticity.


import scipy as sp
test_result, p_value = sp.stats.bartlett(y_pred, residuals)
# To interpret the results we must also compute a critical value of the chi squared distribution

degree_of_freedom = len(y_pred)-1
probability = 1 - p_value
critical_value = sp.stats.chi2.ppf(probability, degree_of_freedom)
print(critical_value)

# If the test_result is greater than the critical value, then we reject our null
# hypothesis. This would mean that there are patterns to the variance of the data
# Otherwise, we can identify no patterns, and we accept the null hypothesis that 
# the variance is homogeneous across our data
if (test_result > critical_value):
  print('the variances are unequal, and the model should be reassessed')
else:
  print('the variances are homogeneous!')

inf
the variances are homogeneous!


# Quantile Regression

In [31]:
import statsmodels.formula.api as smf

# Dividing our data into training and test sets
# ---
# 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# Finding the regression coefficients for the conditioned median, 0.5 quantile
#
mod = smf.quantreg('y ~ X', housedf)
res = mod.fit(q=.5)

# Then print out the summary of our model
#
print(res.summary())

                         QuantReg Regression Results                          
Dep. Variable:                      y   Pseudo R-squared:               0.3706
Model:                       QuantReg   Bandwidth:                   2.525e+04
Method:                 Least Squares   Sparsity:                    3.494e+05
Date:                Thu, 20 Feb 2020   No. Observations:                21610
Time:                        03:55:40   Df Residuals:                    21602
                                        Df Model:                            7
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   4.241e+06   9.59e+04     44.209      0.000    4.05e+06    4.43e+06
X[0]       -2.717e+04   1593.726    -17.048      0.000   -3.03e+04    -2.4e+04
X[1]          -0.0965      0.029     -3.288      0.001      -0.154      -0.039
X[2]         133.6209      2.597     51.449      0.0

/usr/local/lib/python3.6/dist-packages/statsmodels/regression/quantile_regression.py:192: IterationLimitWarning: Maximum number of iterations (1000) reached.
  ") reached.", IterationLimitWarning)


#### Making predictions

In [32]:
 
y_pred = regressor.predict(x_test)

# To compare the actual output values for X_test with the predicted values
# 
quan = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
quan

,Actual,Predicted
20188,602000.0,342260.514727
7573,320000.0,373414.905415
12873,245000.0,378083.784647
209,464000.0,549113.424868
19155,190000.0,140518.290604
...,...,...
11080,368888.0,424518.621360
18910,734200.0,750301.931671
15838,411000.0,587348.015428
8695,410000.0,666443.556808


#### Evaluating the Algorithm


In [33]:

# 
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 148921.50835837692
Mean Squared Error: 57719331207.42589
Root Mean Squared Error: 240248.47805433834


In [34]:
#R2 score
from sklearn.metrics import r2_score
q = r2_score(y_test, y_pred)
q

0.6102276905307119

Our accuracy is just about the same as with the linear model.

60% vs 61%

In [0]:
# Plotting the residual plot
# Residuals have been calculated by by substracting the test value from the predicted value
# 

residuals = np.subtract(y_pred, y_test)
# Plotting the residual scatterplot
#
plt.scatter(y_pred, residuals, color='black')
plt.title('Residual Plot')
plt.ylabel('residual')
plt.xlabel('fitted values')
plt.axhline(y = float(residuals.mean()), color='red', linewidth=1)
plt.show()

# Ridge Regression

In [0]:

# Importing our libraries
# 
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [0]:

# Setting predictors and predicted features
#
x = housedf[['bedrooms', 'bathrooms', 'sqft_above', 'sqft_lot', 'floors','condition', 'grade', 'sqft_above', 'sqft_basement', 'sqft_living15',
'sqft_lot15']]
y = housedf['price']

In [0]:
# Splitting the dataset into training and testing sets
#
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 10)
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [39]:
# Creating our baseline regression model
# This is a model that has no regularization
# 
regression = LinearRegression()
regression.fit(x_train,y_train)
y_pred=regression.predict(x_test)
first_model = (mean_squared_error(y_test, y_pred))
# first_model = (mean_squared_error(y_true=y,y_pred=regression.predict(x_test)))
print(first_model)

62527986238.8567


In [0]:
# determining the most appropriate value for the l2 regularization.
 
ridge = Ridge(normalize=True)
search = GridSearchCV(estimator=ridge,param_grid={'alpha':np.logspace(-5,2,8)},scoring='neg_mean_squared_error',n_jobs=1,refit=True,cv=10)

In [41]:
# We now use the .fit function to run the model and then use the .best_params_ and
#  .best_scores_ function to determine the models strength. 
# 
search.fit(x_train,y_train)
search.best_params_

{'alpha': 0.001}

In [42]:
search.best_score_

-58167585198.66405

In [43]:
# We can confirm this by fitting our model with the ridge information and finding the mean squared error below
#
ridge = Ridge(normalize=True,alpha=0.001)
ridge.fit(x_train,y_train)
second_model = (mean_squared_error(y_true=y_test,y_pred=ridge.predict(x_test)))
print(second_model)

62530948137.29861


In [44]:
# Making predictions
#
y_pred = ridge.predict(x_test)
y_pred
# To compare the actual output values for P_test with the predicted values
# 
r = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
r

,Actual,Predicted
20188,602000.0,409653.342599
7573,320000.0,387038.719921
12873,245000.0,302082.815421
209,464000.0,369729.336364
19155,190000.0,133206.459720
...,...,...
19909,598950.0,425620.090503
19659,535000.0,406838.650963
19153,567000.0,746427.793549
8721,324000.0,314240.924030


In [45]:
# CHecking accuracy using R2 score


from sklearn.metrics import r2_score
r = r2_score(y_test, y_pred)
r

0.5527773032124546

# Lasso Regression

In [0]:
lasso = Lasso(alpha=0.000001, normalize=True)


In [58]:
# Fitting the model

lasso.fit(x_train,y_train)
y_pred2 = lasso.predict(x_test)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 122817611095501.0, tolerance: 216116653881.4755
  positive)


In [59]:
r2_score(y_test,y_pred2)

0.5527984868241287

# Elastic Net Regression

In [0]:
Elastic = ElasticNet(alpha=0.0000001, l1_ratio=0.5, normalize=False)


In [56]:
# Fitting the model

Elastic.fit(x_train,y_train)
y_pred3 = Elastic.predict(x_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 468566609524518.06, tolerance: 216116653881.4755
  positive)


In [57]:
r2_score(y_test,y_pred3)

0.5527984866972273

# Conclusion

Using the models above, we had our accuracy fluctuating between 55% and 61%, with Ridge regression scoring lowest. It is worth noting though, that we included _sqft_living15_ and _sqft_living15_ which weren't present in the other models. This is a classic example of how increasing complexity of the dataset (more features) may not necessarily yield a better accuracy.

This accuracy can be improved by scrutinizing our features more, and deciding what to leave out.

ALso, if we were to assign weights to the different features, anything to do with sq. footage had too much sway compared to the others.